In [1]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import uuid
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

### RQ #2) Which specific flares are the most impactful? 

### RQ #3) Which specific block grous are being most impacted? 


Exploratory question; no hypothesis testing

EJ index = (The Environmental Indicator Percentile for Block Group) x (Demographic Index for Block Group)
  
a) Determine flaring impact metric. 

Something like: buffer_population x EJ Index x flare volume  

b) Calculate impact per flare  

c) Visualize top_x flares by impact  


### Read in files

In [3]:
#ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_counties.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

In [4]:
#ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned
# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp")

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'bg_area'}, inplace=True)  # old:new.
ca_bg_joined.rename(columns={'CNTY_NAME':'cnty_name'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

BGs found: 25607


In [5]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp")

# renaming 
ca_flares.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

print(f'Flares found: {len(ca_flares)}')

Flares found: 117


In [6]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_flares = ca_flares.to_crs(meters_crs)
ca_bg_joined = ca_bg_joined.to_crs(meters_crs)

### Update ca_flares df

In [7]:
# set col list for BCM_avg calculation
bcm_list = ['BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021']

# add new column for average BCM across all years
ca_flares['BCM_avg'] = ca_flares[bcm_list].mean(axis=1) 

In [8]:
# subset to only columns needed for Tableau
col_list = ['flare_id','flare_cate','cnty_name','BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021','BCM_avg', 'geometry']

subset = ca_flares[col_list].copy()

# assign the subsetted copy back to the original variable name to keep total # of df names small 
# throughout project
ca_flares = subset

### Update ca_bg_joined df

In [9]:
# There are five BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# Explore if needed: ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000, 60379903000, 60599901000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['bg_id'].isin(ids_to_drop)]

In [10]:
# subset to only cols needed
subset = ca_bg_joined[['bg_id', 'cnty_name', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'bg_area', 'geometry']].copy()

# assign the subsetted copy back to the original variable name
ca_bg_joined = subset

ca_bg_joined.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

## Create "baseline" df for use in further BG-specific and flare-specific analysis

In [11]:
# set geometry columns
ca_bg_joined = ca_bg_joined.set_geometry('bg_geom')

# and rename for clarity
ca_flares.rename(columns={'geometry':'flarepts_geom'}, inplace=True)  # old:new. Match col names for merging
ca_flares = ca_flares.set_geometry('flarepts_geom')

In [12]:
len(ca_bg_joined['bg_id'].unique())

25602

In [107]:
#buffer_size=2000
buffer_size=5000

In [108]:
buffer_col = f"{buffer_size}_buff_geom"
ca_flares[buffer_col] = ca_flares['flarepts_geom'].buffer(distance=buffer_size)

ca_flares = ca_flares.set_geometry(buffer_col)

# subset of desired columns from flares_df
flares_sjoin_subset = ca_flares[[buffer_col, 'BCM_avg', 'flare_id', 'flare_cate']]

# Left join the flares_sjoin_subset cols to the bg_socialdf based on if any of the flares geometries intersect 
# with the bg geometries
intersect_baseline = gp.sjoin(ca_bg_joined, flares_sjoin_subset, how='left', predicate='intersects')

# Bring in buffer col that is not brought in by the sjoin() function for some unknown reason. Frustrating.
intersect_baseline = pd.merge(intersect_baseline, flares_sjoin_subset[['flare_id', buffer_col]], 
                             on='flare_id', how='left')

# set geom to the buffers and filter df down to just these BGs that have a flare_id associated with them. 
# There's no need to calculate any of the following for non-intersected BGs. 
intersect_baseline = intersect_baseline.set_geometry(buffer_col)
intersect_baseline = intersect_baseline[intersect_baseline.is_valid].copy()
print(f'{buffer_size}m BG-Buffer intersections: {len(intersect_baseline)}')

5000m BG-Buffer intersections: 3002


In [109]:
# Iterate through each bg_id and flare_id pairing and calculate 
# the proportion of the buffer that intersects with the BG geom, relative to the total buffer size.
# We need to know the proportion of each buffer in each bg geom in order to assign the percentage of
# flare volume to each bg in the case of incomplete overlap.

# First check for null or invalid geometries in buffer_2000m column
intersect_baseline['buffer_valid'] = intersect_baseline[buffer_col].is_valid

# Initialize intersect proportion "b_int_prop" and intersect area "b_int_area" columns with NaN values
intersect_baseline[f'{buffer_size}_buff_int_prop'] = np.nan
intersect_baseline[f'{buffer_size}_buff_int_area'] = np.nan

# Loop through each block group and flare pairing
for index, row in intersect_baseline.iterrows():
    # Check if buffer_2000m geometry is valid
    if row['buffer_valid']:
        # Calculate intersection area
        intersection = row[buffer_col].intersection(row['bg_geom'])  # prop of buffer in the bg geom
        intersection_area = intersection.area
        # Calculate buffer area
        buffer_area = row[buffer_col].area
        # Calculate intersection proportion
        buffer_int_prop = intersection_area / buffer_area
        # Update int_prop and int_area columns with calculated values
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = buffer_int_prop
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = intersection_area
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = buffer_area

    else:
        # Assign 0 to b_int_prop and b_int_area if buffer_2000m geometry is invalid
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = 0


# Drop buffer_valid column as it's no longer needed
intersect_baseline.drop('buffer_valid', axis=1, inplace=True)

In [110]:
# Now a similar analysis but this time we want to know the BG-buffer intersection proportion relative to 
# the total area of the block group, in order to use that proportion to assign counts of BG demographic
# variables to the intersection area. 

# Combine all flares into a single multipolygon unit
temp = ca_flares.unary_union
all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=ca_flares.crs) # convert back to geodf for processing

intersect_baseline = intersect_baseline.set_geometry('bg_geom')

# Make sure our geom column is set correctly to "bg_geom" rather than the flare buffer
if intersect_baseline.geometry.name == "bg_geom":
    int_temp = gp.overlay(intersect_baseline, all_buffers, how='intersection')  

    int_temp.rename(columns={'geometry':f'{buffer_size}_bg_int_geom'}, inplace=True)  # old:new. For clarity

    # Set geometry to the intersections
    int_temp = int_temp.set_geometry(f'{buffer_size}_bg_int_geom')  

    # Create new 'area' column for the areas of the intersections
    int_temp[f'{buffer_size}_bg_int_area'] = int_temp.area

    # Calculate the proportion of each block group intersection relative to total block broup area
    int_temp[f'{buffer_size}_bg_int_prop'] = int_temp[f'{buffer_size}_bg_int_area'] / int_temp['bg_area']

    # Merge the new area and props variables
    intersect_baseline = pd.merge(intersect_baseline, 
                                 int_temp[['bg_id',
                                           f'{buffer_size}_bg_int_prop',
                                           f'{buffer_size}_bg_int_area', 
                                           f'{buffer_size}_bg_int_geom']], 
                                 on=['bg_id'], 
                                 how='left').fillna({f'{buffer_size}_bg_int_prop':0, 
                                                     f'{buffer_size}_bg_int_area':0})

else:
    print("Error: Geometry column is not named 'bg_geom'")

In [111]:
# Apply the proportion to each demographic variable to find counts by variable
# i.e. this answers "how many of x live in the X buffer radius intersection area?"
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    intersect_baseline[f'{buffer_size}_{var}_bg_int_count'] = (intersect_baseline[var] *
                                                               intersect_baseline[f'{buffer_size}_bg_int_prop'])

# Find *overall* proportions for each demo var by dividing var count by respective total population
# i.e. this answers "what overall percentage of this block group is minority status?"
intersect_baseline['MINORPOP_bg_totprop'] = intersect_baseline['MINORPOP'] / intersect_baseline['ACSTOTPOP']

# calculate percentage of minorities in intersection for each BG
intersect_baseline[f'{buffer_size}_bg_int_mnr_prop'] = (intersect_baseline[f'{buffer_size}_MINORPOP_bg_int_count']
                                                        / intersect_baseline[f'{buffer_size}_ACSTOTPOP_bg_int_count'])

# rename the flare categories for readability
intersect_baseline["flare_cate"] = (intersect_baseline["flare_cate"].
                                    replace({"flares_oil_downstream": "down_oil",
                                             "flares_upstream": "upstream"}))

In [112]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 
df_baseline_all = intersect_baseline.drop_duplicates(subset=['bg_id', 'flare_id'])

In [113]:
len(df_baseline_all)

3002

In [114]:
# grab two specific bgs to check
df_baseline_all[df_baseline_all['bg_id'].isin([60379800331, 60379800111])]


,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,5000_buff_geom,5000_buff_int_prop,5000_buff_int_area,5000_buff_tot_area,5000_bg_int_prop,5000_bg_int_area,5000_bg_int_geom,5000_ACSTOTPOP_bg_int_count,5000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,5000_bg_int_mnr_prop
567,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",113.00,0.00,857477a0194312,down_oil,"POLYGON ((170197.511 -469609.028, 170173.435 -...",0.02,"1,376,685.28","78,413,712.26",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
573,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",106.00,0.00,b30e8627682313,down_oil,"POLYGON ((169954.446 -469593.965, 169930.370 -...",0.02,"1,376,685.28","78,413,712.26",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
579,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",107.00,0.00,caf4a94c957987,down_oil,"POLYGON ((170134.921 -469587.135, 170110.845 -...",0.02,"1,376,685.28","78,413,712.26",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
585,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",108.00,0.00,f3188e83798361,down_oil,"POLYGON ((170283.806 -469521.668, 170259.730 -...",0.02,"1,376,685.28","78,413,712.26",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
591,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",112.00,0.00,34208bde510417,down_oil,"POLYGON ((170197.510 -469510.426, 170173.433 -...",0.02,"1,376,685.28","78,413,712.26",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
597,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",111.00,0.00,90207aa4167979,down_oil,"POLYGON ((168836.943 -465162.056, 168812.867 -...",0.02,"1,376,685.28","78,413,712.26",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
2987,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",113.00,0.00,857477a0194312,down_oil,"POLYGON ((170197.511 -469609.028, 170173.435 -...",0.29,"22,536,997.15","78,413,712.26",0.79,"22,601,128.56","POLYGON ((162259.677 -471740.729, 162351.365 -...",11.05,11.05,1.00,1.00
2993,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",106.00,0.00,b30e8627682313,down_oil,"POLYGON ((169954.446 -469593.965, 169930.370 -...",0.28,"22,300,202.00","78,413,712.26",0.79,"22,601,128.56","POLYGON ((162259.677 -471740.729, 162351.365 -...",11.05,11.05,1.00,1.00
2999,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",107.00,0.00,caf4a94c957987,down_oil,"POLYGON ((170134.921 -469587.135, 170110.845 -...",0.29,"22,392,544.58","78,413,712.26",0.79,"22,601,128.56","POLYGON ((162259.677 -471740.729, 162351.365 -...",11.05,11.05,1.00,1.00
3005,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",108.00,0.00,f3188e83798361,down_oil,"POLYGON ((170283.806 -469521.668, 170259.730 -...",0.28,"22,139,256.48","78,413,712.26",0.79,"22,601,128.56","POLYGON ((162259.677 -471740.729, 162351.365 -...",11.05,11.05,1.00,1.00


# Aggregate data by BG and export files for Tableau

take baseline dfs and create copies clearly defined for this BG analysis, separate from the work that is needed
for the flare buffer analysis below. Both analyses modify these same two dfs in different ways and I want 
to make sure I keep the calculations separated

#### Standardize variables

EJ index and total population in the intersection(s) are already encoded at the level of a BG, so they're ready to normalize. 

Flare volume needs to multiplied by the proportion of the flare's buffer zone that resides in a BG, and then all (in the case of multiple flares in a BG) adjusted flare volumes summed up to generate a variable for total flare volume a BG is exposed to.  

In [41]:
def bg_calc_adj_flare_vol(df, buffer_size):
    """
    Function for calculating total flare volume to be assigned to each BG, based on amount of overlap.
    
    "bcm_adj" = adjusted bcm value for a given flare_id and bg_id, based on the proportion of the buffer zone that 
    overlaps with a given bg_id. Equal to the bcm_avg * b_int_prop (the overlap)
    
    "bcm_adj_tot" = the sum of the bcm_adj values for a given bg_id
    """
    
    # calculate the product of bcm_avg and buff_int_prop for each flare and bg combination
    df[f'{buffer_size}_bcm_int_adj'] = df['BCM_avg'] * df[f'{buffer_size}_buff_int_prop']
    
    # sum the adjusted volumes for all flare and bg combinations
    adj_vol = df.groupby(['flare_id', 'bg_id'])[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # sum those adjusted volumes for all flares in each block group
    bg_vol = adj_vol.groupby('bg_id')[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # merge the total adjusted block group volumes with the original dataframe
    df = pd.merge(df, bg_vol, on='bg_id', how='left', suffixes=('', '_tot'))
    
    return df

In [42]:
df_bg_intersect = df_baseline_all.groupby('bg_id').apply(bg_calc_adj_flare_vol, buffer_size=buffer_size)

In [43]:
# Define the variables to standardize

#'bcm_adj_tot' = Sum of the adjusted flare volume(s) that intersect w the BG. 
#'D_PM25_2' = EJ index for the BG
#'ACSTOTPOP_int_count' = BG population that resides in the intersection. = BGtotpop * intersection_proportion

varlist = [f'{buffer_size}_bcm_int_adj_tot', 'D_PM25_2', f'{buffer_size}_ACSTOTPOP_bg_int_count']

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(df_bg_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

intersect_norm = intersect_norm.reset_index(drop=True)
df_bg_intersect = df_bg_intersect.reset_index(drop=True)

# Concatenate the new variables with the original DataFrame
df_bg_intersect = pd.concat([df_bg_intersect, intersect_norm], axis=1)

In [44]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 
df_bg_intersect = df_bg_intersect.drop_duplicates(subset=['bg_id', 'flare_id'])

In [45]:
len(df_bg_intersect)

3002

In [46]:
# # check calculations against two BGs
# # Should see a single "bcm_adj_tot" value per BG, and different adjusted "bcm_adj" values per flare_id 
# #depending on the amount of overlap.

# # filter the dataframe and format specified columns to display 4 decimal places
# filtered_df = df_bg_intersect_all[df_bg_intersect_all['bg_id'].isin([60379800331, 60379800111])].copy()
# filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj', f'{buffer_size}_bcm_int_adj_tot', 
#              f'{buffer_size}_bcm_int_adj_tot_n']] = filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj',
#                                                                  f'{buffer_size}_bcm_int_adj_tot', 
#                                                                  f'{buffer_size}_bcm_int_adj_tot_n']].apply(lambda x: x.apply(lambda y: '{:.4f}'.format(y)))


In [47]:
# filtered_df

### Split geom data from rest of data b/c column names must be <10 characters for shapefiles and that makes naming my different columns unnecessarily confusing. I export a csv of everything but the geometry, export a shapefile of just the geometry, then join the two back together in Tableau. 

In [48]:
df_bg_intersect.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,5000_buff_geom,5000_buff_int_prop,5000_buff_int_area,5000_buff_tot_area,5000_bg_int_prop,5000_bg_int_area,5000_bg_int_geom,5000_ACSTOTPOP_bg_int_count,5000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,5000_bg_int_mnr_prop,5000_bcm_int_adj,5000_bcm_int_adj_tot,5000_bcm_int_adj_tot_n,D_PM25_2_n,5000_ACSTOTPOP_bg_int_count_n
2087,60590626144,Orange County,4208,3030,78.93,"362,087.30","POLYGON ((201340.459 -482808.734, 201382.529 -...",88.00,0.00,7f8095b7782755,upstream,"POLYGON ((207125.440 -486756.777, 207101.364 -...",0.00,"362,087.30","78,413,712.26",1.00,"362,087.30","POLYGON ((201340.459 -482808.734, 201382.529 -...","4,208.00","3,030.00",0.72,0.72,0.00,0.00,-0.20,1.43,3.82


### 2000

In [30]:
df_bgintersect_geo2000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '2000_bg_int_geom']], 
                                         geometry='2000_bg_int_geom')

print(len(df_bgintersect_geo2000[df_bgintersect_geo2000.is_valid]))

df_bgintersect_2000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', f'{buffer_size}_bg_int_geom'], axis=1)

501


In [31]:
df_bgintersect_2000.head(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n
0,60133132031,Contra Costa County,729,495,33.27,"514,270.79",89.00,0.01,342ab9d4310483,upstream,0.04,"513,196.18","12,546,193.96",1.00,"513,196.18",727.48,493.97,0.68,0.68,0.00,0.00,-0.28,-0.50,0.19
1,60133132031,Contra Costa County,729,495,33.27,"514,270.79",90.00,0.01,47291ae3406417,upstream,0.04,"512,561.56","12,546,193.96",1.00,"513,196.18",727.48,493.97,0.68,0.68,0.00,0.00,-0.28,-0.50,0.19


In [32]:
df_bgintersect_geo2000.to_file('data/df_bg_geo2000.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### 5000

In [49]:
# Create a new geodataframe with just the id and geom columns
df_bgintersect_geo5000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '5000_bg_int_geom']], 
                                         geometry='5000_bg_int_geom')

print(len(df_bgintersect_geo5000[df_bgintersect_geo5000.is_valid]))


# Subset the original dataframe and drop unnecessary columns
df_bgintersect_5000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', 
                                         f'{buffer_size}_bg_int_geom'], axis=1)

3002


In [50]:
df_bgintersect_5000.head(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,5000_buff_int_prop,5000_buff_int_area,5000_buff_tot_area,5000_bg_int_prop,5000_bg_int_area,5000_ACSTOTPOP_bg_int_count,5000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,5000_bg_int_mnr_prop,5000_bcm_int_adj,5000_bcm_int_adj_tot,5000_bcm_int_adj_tot_n,D_PM25_2_n,5000_ACSTOTPOP_bg_int_count_n
0,60133090001,Contra Costa County,1518,922,47.45,"2,295,826.27",89.00,0.01,342ab9d4310483,upstream,0.01,"416,437.32","78,413,712.26",0.18,"416,442.27",275.35,167.24,0.61,0.61,0.00,0.00,-0.19,-0.00,-1.21
1,60133090001,Contra Costa County,1518,922,47.45,"2,295,826.27",90.00,0.01,47291ae3406417,upstream,0.01,"412,831.10","78,413,712.26",0.18,"416,442.27",275.35,167.24,0.61,0.61,0.00,0.00,-0.19,-0.00,-1.21


In [51]:
df_bgintersect_geo5000.to_file('data/df_bg_geo5000.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### Combine

In [52]:
# filter the second dataframe to only include columns starting with '5000_' plus the ID columns
# all the non "5000_" columns in the second dataframe are identical to the first df, so can be ignored.
# select columns that start with '5000_' and keep 'bg_id' and 'flare_id'
cols_to_keep = ['bg_id', 'flare_id'] + [col for col in df_bgintersect_5000.columns if col.startswith('5000_')]

In [53]:
df_bgintersect_5000_filter = df_bgintersect_5000[cols_to_keep].copy()

In [55]:
# merge the two dataframes on 'bg_id' and 'flare_id', keeping all columns from the first dataframe
df_bg_intersect_all = pd.merge(df_bgintersect_2000, df_bgintersect_5000_filter, on=['bg_id', 'flare_id'], how='outer')

In [56]:
df_bg_intersect_all.to_csv('data/df_bg_impact_all.tsv', sep='\t', index=False)

# Aggregate by Flare ID and export files for Tableau

Take baseline dfs and create copies clearly defined for this flare buffer analysis, 
separate from the work that is needed for the BG buffer analysis above. 
Both analyses modify these same two dfs in different ways; need to ensure I keep the calculations separated.
Plus, they'll go into Tableau as different dataframes anyway. 

In [115]:
# Use the baseline df as the starting point

df_baseline_all.sample()

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,5000_buff_geom,5000_buff_int_prop,5000_buff_int_area,5000_buff_tot_area,5000_bg_int_prop,5000_bg_int_area,5000_bg_int_geom,5000_ACSTOTPOP_bg_int_count,5000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,5000_bg_int_mnr_prop
5534,60375723012,Los Angeles County,1540,1431,65.64,"462,945.52","POLYGON ((164374.916 -464116.346, 164377.508 -...",107.00,0.00,caf4a94c957987,down_oil,"POLYGON ((170134.921 -469587.135, 170110.845 -...",0.00,"161,005.71","78,413,712.26",1.00,"462,945.52","POLYGON ((164374.916 -464116.346, 164377.508 -...","1,540.00","1,431.00",0.93,0.93


In [116]:
# Define the variables to aggregate

# BCM_avg: No adjustments needed because bcm is already attached to the flare unit of analysis
# D_PM25_D: Create new var of avg PM25 score for all BGs in each buffer 
# ACSTOTPOP_bg_int_count: Create new var of total population in all intersections in each buffer

agg_dict = {'D_PM25_2': 'mean', f'{buffer_size}_ACSTOTPOP_bg_int_count': 'sum'}

# set up df for flare-specific calcs
flare_intersect = df_baseline_all.copy()

# Group the block groups by flare ID and aggregrate the selected vars
flare_agg = df_baseline_all.groupby('flare_id').agg(agg_dict)

# Rename the output variables
flare_agg = flare_agg.rename(columns={'D_PM25_2': f'{buffer_size}_int_pm25_avg', 
                                      f'{buffer_size}_ACSTOTPOP_bg_int_count': f'{buffer_size}_int_pop_sum'})

# Join the aggregated variables with the original columns
flare_intersect = flare_intersect.merge(flare_agg, on='flare_id')


In [117]:
flare_intersect.sample(3)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,5000_buff_geom,5000_buff_int_prop,5000_buff_int_area,5000_buff_tot_area,5000_bg_int_prop,5000_bg_int_area,5000_bg_int_geom,5000_ACSTOTPOP_bg_int_count,5000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,5000_bg_int_mnr_prop,5000_int_pm25_avg,5000_int_pop_sum
95,60376031013,Los Angeles County,1213,867,55.38,"278,366.69","POLYGON ((157329.199 -457042.621, 157328.771 -...",110.00,0.00,36c62e67595483,down_oil,"POLYGON ((159983.129 -460981.588, 159959.053 -...",0.00,"194,760.48","78,413,712.26",1.00,"278,366.69","POLYGON ((157329.199 -457042.621, 157328.771 -...","1,213.00",867.00,0.71,0.71,45.67,"268,503.66"
2551,61110012041,Ventura County,2831,1652,29.14,"120,975,492.62","POLYGON ((63479.061 -405864.925, 63480.089 -40...",20.00,0.00,845ed79f581833,upstream,"POLYGON ((67526.668 -410777.691, 67502.591 -41...",0.14,"11,293,713.95","78,413,712.26",0.27,"32,108,152.35","POLYGON ((63479.061 -405864.925, 63480.089 -40...",751.38,438.46,0.58,0.58,28.50,"19,190.35"
862,60375433051,Los Angeles County,3808,3214,53.01,"8,386,713.67","POLYGON ((163213.384 -458668.316, 163233.820 -...",111.00,0.00,90207aa4167979,down_oil,"POLYGON ((168836.943 -465162.056, 168812.867 -...",0.05,"3,538,524.70","78,413,712.26",0.42,"3,538,524.70","POLYGON ((166155.538 -460734.293, 166150.732 -...","1,606.67","1,356.05",0.84,0.84,57.92,"223,544.70"


In [118]:
# Define the variables to standardize

varlist = ['BCM_avg', f'{buffer_size}_int_pm25_avg', f'{buffer_size}_int_pop_sum']  

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(flare_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

# Concatenate the new variables with the original DataFrame
flare_intersect = pd.concat([flare_intersect, intersect_norm], axis=1)

In [119]:
type(flare_intersect)

geopandas.geodataframe.GeoDataFrame

In [120]:
flare_intersect.sample(3)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,5000_buff_geom,5000_buff_int_prop,5000_buff_int_area,5000_buff_tot_area,5000_bg_int_prop,5000_bg_int_area,5000_bg_int_geom,5000_ACSTOTPOP_bg_int_count,5000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,5000_bg_int_mnr_prop,5000_int_pm25_avg,5000_int_pop_sum,BCM_avg_n,5000_int_pm25_avg_n,5000_int_pop_sum_n
2947,60190078021,Fresno County,644,594,74.42,"413,398,121.61","POLYGON ((-19002.176 -188418.954, -19002.053 -...",77.00,0.00,d5158879356806,upstream,"POLYGON ((-7930.506 -175532.902, -7954.583 -17...",0.39,"30,381,801.10","78,413,712.26",0.09,"35,486,552.16","POLYGON ((-17351.469 -178135.180, -17352.796 -...",55.28,50.99,0.92,0.92,68.58,335.69,-0.55,1.29,-1.31
676,60376203013,Los Angeles County,1159,200,12.77,"259,900.43","POLYGON ((146835.707 -455817.956, 147097.697 -...",109.00,0.00,c51a879d128139,down_oil,"POLYGON ((150272.316 -453153.575, 150248.239 -...",0.00,"259,900.43","78,413,712.26",1.00,"259,900.43","POLYGON ((146835.707 -455817.956, 147097.697 -...","1,159.00",200.00,0.17,0.17,20.98,"65,979.00",-0.55,-1.63,-0.69
2468,60710021091,San Bernardino County,1491,1342,66.10,"248,396.05","POLYGON ((223254.764 -435234.263, 223256.342 -...",116.00,0.00,ade49180235149,upstream,"POLYGON ((230403.428 -439194.020, 230379.352 -...",0.00,"248,396.05","78,413,712.26",1.00,"248,396.05","POLYGON ((223254.764 -435234.263, 223256.342 -...","1,491.00","1,342.00",0.90,0.90,52.27,"39,797.07",-0.53,0.29,-0.94


### 2000

In [105]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flare_intersect_geo2000 = gp.GeoDataFrame(flare_intersect[['bg_id', 'flare_id', '2000_bg_int_geom']].copy(), 
                                      geometry='2000_bg_int_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flare_intersect_2000 = flare_intersect.drop(['bg_geom', '2000_bg_int_geom', '2000_buff_geom'], axis=1)

In [106]:
flare_intersect_geo2000.to_file("data/df_flareimp_geo2000.shp", driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### 5000

In [121]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flare_intersect_geo5000 = gp.GeoDataFrame(flare_intersect[['bg_id', 'flare_id', '5000_bg_int_geom']].copy(), 
                                      geometry='5000_bg_int_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flare_intersect_5000 = flare_intersect.drop(['bg_geom', '5000_bg_int_geom', '5000_buff_geom'], axis=1)

In [122]:
flare_intersect_geo5000.to_file("data/df_flareimp_geo5000.shp", driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### Combine

In [123]:
# filter the second dataframe to only include columns starting with '5000_' plus the ID columns
# all the non "5000_" columns in the second dataframe are identical to the first df, so can be ignored.
# select columns that start with '5000_' and keep 'bg_id' and 'flare_id'
flarecols_to_keep = (['bg_id', 'flare_id'] + 
                     [col for col in flare_intersect_5000.columns if col.startswith('5000_')])

In [124]:
flare_intersect_5000_filter = flare_intersect_5000[flarecols_to_keep].copy()

In [125]:
# merge the two dataframes on 'bg_id' and 'flare_id', keeping all columns from the first dataframe
df_flare_intersect_all = pd.merge(flare_intersect_2000, flare_intersect_5000_filter, 
                               on=['bg_id', 'flare_id'], how='outer')

In [126]:
df_flare_intersect_all.to_csv('data/df_flare_impact_all.tsv', sep='\t', index=False)

## Calculating single instance of weights here in python

In [ ]:
# # Define the weights for each variable
# bcm_weight = 0
# pm25_weight = 1
# pop_weight = 0

In [ ]:
# # Calculate the weighted variables
# intersect['BCM_weighted'] = intersect['BCM_avg_norm'] * bcm_weight
# intersect['D_PM25_2_weighted'] = intersect['D_PM25_2_norm'] * pm25_weight
# intersect['ACSTOTPOP_weighted'] = intersect['ACSTOTPOP_intersect_count_norm'] * pop_weight

# # Define the variables to sum
# varlist_weighted = ['BCM_weighted', 'D_PM25_2_weighted', 'ACSTOTPOP_weighted']

# # Group the block groups by flare ID and sum the weighted variables
# ca_flares_merged = intersect.groupby('flare_id')[varlist_weighted].sum()

# # Calculate the impact metric as the sum of the weighted variables
# ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_weighted].sum(axis=1)

# # Sort the flares by impact metric in descending order
# ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [ ]:
# # Show the results
# ca_flares_merged.head()

In [ ]:
# # Define the variables to sum
# varlist_norm = ['BCM_avg_norm', 'D_PM25_2_norm', 'ACSTOTPOP_intersect_count_norm']

# # Group the block groups by flare ID and sum the standardized variables
# ca_flares_merged = intersect.groupby('flare_id')[varlist_norm].sum()

# # Calculate the impact metric as the sum of the standardized variables
# ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_norm].sum(axis=1)

# # Sort the flares by impact metric in descending order
# ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [ ]:
# df_final = pd.merge(ca_flares_merged, intersect, on='flare_id')

In [ ]:
# invalid_geoms = df_final[~df_final.is_valid]

In [ ]:
# invalid_geoms

In [ ]:
# # save for use in tableau
# df_final.to_csv(F"data/df_impactmetric_csv.csv", index=False)

In [ ]:
# # check for missing values
# missing_values = df_final.isnull().sum()

# # filter columns with missing values
# missing_cols = missing_values[missing_values > 0]

# # print column names and number of missing values
# for col in missing_cols.index:
#     print(f"Column '{col}' has {missing_cols[col]} missing values")

In [ ]:
# no_missing = missing_values[missing_values == 0]
# # print column names and number of missing values
# for col in no_missing.index:
#     print(f"Column '{col}' has {no_missing[col]} missing values")

In [ ]:
# df_final.sample(2)

## Top ten

In [ ]:
# # display the top ten flares by impact metric
# top_ten = for_map.nlargest(10, 'impact_metric')
# top_ten = top_ten.set_geometry('buffer_2000m')

In [ ]:
# # check for missing values
# missing_values = for_map.isnull().sum()

# # filter columns with missing values
# missing_cols = missing_values[missing_values > 0]

# # print column names and number of missing values
# for col in missing_cols.index:
#     print(f"Column '{col}' has {missing_cols[col]} missing values")

In [ ]:
# no_missing = missing_values[missing_values == 0]
# # print column names and number of missing values
# for col in no_missing.index:
#     print(f"Column '{col}' has {no_missing[col]} missing values")

## Folium Mapping

In [ ]:
# for_map = df_final[['flare_id', 'BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count', 'impact_metric', 'buffer_2000m']]
# for_map = gp.GeoDataFrame(for_map, geometry='buffer_2000m', crs=meters_crs)

# for_map.to_file("data/df_impactmetric_shp.shp", driver='ESRI Shapefile')

In [ ]:

# # Define the color scale and number of bins
# #color_scale = 'Reds'
# #num_bins = 10

# # Create a map centered on the first flare
# # Create a folium map with a center location
# m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
# base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
# folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
# base_map.add_to(m)




In [ ]:
# # Feature Group: Buffers

# def style_function3(feature):
#     return {
#         'fillColor': 'red',
#         'color': 'red',
#         'fillOpacity': 0.05
#     }

# inter_all_bg = folium.FeatureGroup(name='Intersect BG', overlay=True)
# folium.GeoJson(
#     data=test["buffer_2000m"],
#     style_function=style_function3
# ).add_to(inter_all_bg)
# inter_all_bg.add_to(m)

In [ ]:
# # Feature Group: block group

# def style_function2(feature):
#     return {
#         'fillColor': 'blue',
#         'color': 'blue',
#         'fillOpacity': 0.05
#     }

# bg_bound = folium.FeatureGroup(name='Intersect BG', overlay=True)
# folium.GeoJson(
#     data=test["bg_geom"],
#     style_function=style_function2
# ).add_to(bg_bound)
# bg_bound.add_to(m)

In [ ]:
# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#999999', 
#                                 'color':'#999999', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}
# NIL = folium.features.GeoJson(
#     data = for_map,
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['flare_id', 'BCM_avg_norm', 'D_PM25_2_norm', 
#              'ACSTOTPOP_intersect_count_norm', 'impact_metric'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )

In [ ]:
# # add hover functionality as child to map, add layering, display map
# m.add_child(NIL)
# m.keep_in_front(NIL)
# folium.LayerControl().add_to(m)

In [ ]:
# # Display the map
# m